In [1]:
## Requiered packages

%pip install pymupdf python-docx transformers 


     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.8 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 43.8/43.8 kB 430.5 kB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   - -------------------------------------- 0.2/9.1 MB 5.0 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/9.1 MB 6.0 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/9.1 MB 8.0 MB/s eta 0:00:01
   ------- -------------------------------- 1.7/9.1 MB 9.0 MB/s eta 0:00:01
   --------- ------------------------------ 2.3/9.1 MB 9.6 MB/s eta 0:00:01
   ------------ --------------------------- 2.8/9.1 MB 10.0 MB/s eta 0:00:01
   -------------- ------------------------- 3.4/9.1 MB 10.3 MB/s eta 0:00:01
   ----------

In [2]:
%pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install chromadb


Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install torch

   ---------------------------------------- 0.0/159.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/159.7 MB 3.3 MB/s eta 0:00:49
   ---------------------------------------- 0.4/159.7 MB 5.0 MB/s eta 0:00:32
   ---------------------------------------- 0.9/159.7 MB 7.4 MB/s eta 0:00:22
   ---------------------------------------- 1.5/159.7 MB 8.0 MB/s eta 0:00:20
    --------------------------------------- 2.1/159.7 MB 8.8 MB/s eta 0:00:18
    --------------------------------------- 2.7/159.7 MB 9.4 MB/s eta 0:00:17
    --------------------------------------- 3.1/159.7 MB 10.0 MB/s eta 0:00:16
    --------------------------------------- 3.7/159.7 MB 10.2 MB/s eta 0:00:16
   - -------------------------------------- 4.3/159.7 MB 10.2 MB/s eta 0:00:16
   - -------------------------------------- 4.9/159.7 MB 10.3 MB/s eta 0:00:15
   - -------------------------------------- 5.4/159.7 MB 10.5 MB/s eta 0:00:15
   - -------------------------------------- 6.0/159.7 MB 10.7 MB/s

In [2]:
## Importing the required libraries

from langchain_community.document_loaders import PyPDFLoader
from docx import Document
from transformers import AutoTokenizer, AutoModel
import chromadb
from chromadb.utils import embedding_functions
import numpy as np
import torch
import os


In [6]:
# # Setting up iopenai api key

# from dotenv import load_dotenv
# load_dotenv()

# openai.api_key = os.getenv("OPENAI_API_KEY")


In [3]:
def read_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load_and_split()
    text = " ".join([doc.page_content for doc in documents])
    # print(text)
    return text

# read_pdf("C:/Users/JineshPatel/Downloads/Get_Started_With_Smallpdf.pdf")

In [4]:
def read_doc(file_path):
    doc = Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text
    return text


In [9]:
# import json

# def generate_embeddings(text):
#     text_str = str(text)

#     response = openai.Embedding.create(input=json.dumps(text_str), model="text-embedding-ada-002")
#     embeddings = response['data'][0]['embedding']
#     return np.array(embeddings)


# Load a pre-trained model and tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def generate_embeddings(text):
    # print(text ,  type(text))
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()


In [10]:
client = chromadb.Client()
collection = client.create_collection("document_embeddings")

# def index_document(file_path, doc_type):
#     if doc_type == "pdf":
#         text = read_pdf(file_path)
#     elif doc_type == "doc":
#         text = read_doc(file_path)
    
#     embedding = generate_embeddings(text)
#     collection.add({"id": file_path, "embedding": embedding.tolist(), "metadata": {"text": text}})


In [32]:
def query_chromadb(question, n_results=5):
    question_embedding = generate_embeddings(question)
    results = collection.query(
        query_embeddings=[question_embedding.tolist()],
        n_results=n_results
    )
    return results

def retrieve_documents(results):
    documents = results["documents"]
    return [doc["metadata"]["text"] for doc in documents]

# def generate_formatted_answer(question, documents):
#     context = " ".join(documents)
#     response = openai.Completion.create(
#         engine="text-davinci-003",
#         prompt=f"Question: {question}\n\nContext: {context}\n\nAnswer:",
#         max_tokens=200
#     )
#     answer = response.choices[0].text.strip()
#     return answer


from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

def generate_answer(question, documents):
    context = " ".join(documents)
    answer = qa_pipeline(question=question, context=context)
    return answer


def process_question(question):
    results = query_chromadb(question)
    documents = retrieve_documents(results)
    formatted_answer = generate_answer(question, documents)
    return formatted_answer


In [12]:
# %pip install pypdf


In [33]:
def index_document(file_path, doc_type):
    if doc_type == "pdf":
        text = read_pdf(file_path)
    elif doc_type == "doc":
        text = read_doc(file_path)
    else:
        raise ValueError("Unsupported document type")
    
    embedding = generate_embeddings(text)
    collection.add(
        ids=[file_path], 
        embeddings=[embedding.tolist()], 
        metadatas=[{"text": text}]
    )

# Index a document
index_document("C:/Users/JineshPatel/Downloads/Budget_2024.pdf", "pdf")

# Process a question
question = "what is Annual Financial Statement "


Insert of existing embedding ID: C:/Users/JineshPatel/Downloads/Budget_2024.pdf
Add of existing embedding ID: C:/Users/JineshPatel/Downloads/Budget_2024.pdf


In [34]:
answer = process_question(question)
print(answer)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


TypeError: list indices must be integers or slices, not str